
# GPU Numba and CuPy Parallelization of Matrix Multiplication 

Similary to the multicore parallelization lab, in this lab we will be using Numba and CuPy to accelerate matrix-matrix multiplications using GPU. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba for CUDA](https://numba.readthedocs.io/en/stable/cuda/index.html)
* [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb)
* [Numba.CUDA by Graham Markell](https://github.com/numba/nvidia-cuda-tutorial)
* [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/)
* [CuPy Basics](https://docs.cupy.dev/en/stable/user_guide/basic.html)

[//]: <> (GEOPHYS 257 Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

<br>

### Exercise 0

* You need to request a *T4* node on the cluster. Don't forget that you need to add **--gres=gpu** to your srun command.
* Reminder: on the *T4* nodes you need to load a different version of Python:
```bash
spack load python@3.10.7
```

* Import every Python module, object, and/or function that you need below.

In [1]:
from numba import cuda
import numpy as np
from time import time
import cupy

print(cuda.gpus)
print()
!nvcc -V

<Managed Device 0>

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


<br>

### Exercise 1: Matrix Transpose

Before we examine matrix-matrix multiplication, we will first write a GPU kernel that transposes a square matrix.  This type of problem is a good introduction into how to use the CUDA threading model. The task for this exercise is to write a Numba CUDA kernel that will transpose a square matrix. 

**Before you start**, take a look at the following:
* Read over the following notebook that explains Numba.CUDA kernels: [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb) 
* The first matrix-matrix multiplication code (the one that **doesn't** use shared memory) shown at [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). Understanding this code should give a pretty good idea on how to write the transpose kernel. The matrix-matrix kernel code from the NYU lab is shown below.
```python
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
```

**Tasks for this exercise**
* Write a Numba.CUDA kernel that transpose an $NxN$ square matrix.
* Be sure that the transpose kernel can transpose square matrices with sizes of $N$ as small as $N=2$ and as large as $N=10240$.
* Using shared memory is **not** required.


In [2]:
@cuda.jit
def mat_trans(A, AT):
    """ Perform matrix transpose of A
    """
    
    start_x, start_y = cuda.grid(2)
    stride_x, stride_y = cuda.gridsize(2)

    for i in range(start_x, A.shape[0], stride_x):
        for j in range(start_y, A.shape[1], stride_y):
            AT[j, i] = A[i, j]

In [3]:
# Matrix size and float type
N = 10240
ftype = 'float64'

# Create random matrix and copy to device
h_A = np.random.random((N, N)).astype(ftype)
d_A = cuda.to_device(h_A)

# Allocate memory on device for transpose
d_AT = cuda.device_array((N, N), dtype=ftype)

# Configure the blocks
max_threads_per_block = 16
max_blocks_per_grid = 12

threads_per_block = np.min([max_threads_per_block, N])
blocks_per_grid = np.min([max_blocks_per_grid, N//threads_per_block])

# Cuda kernel for transpose
mat_trans[(blocks_per_grid, blocks_per_grid), (threads_per_block, threads_per_block)](d_A, d_AT)

# Copy result back to host
h_AT = d_AT.copy_to_host()

# Check result
assert np.all(h_AT == h_A.T), 'Result not correct!'

<br>

### Exercise 2: Using Numba CUDA to parallelize matrix multiplication: 

For this exercise, we will use Numba compiled GPU kernels that calculate matrix-matrix multiplication for square matrices. In particular, we will use a GPU kernel that doesn't used shared memory and compared to a GPU kernel that does use shared-memory. Please use the two kernel codes discussed in the following lab: [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). As you will see in this exercise, learning to use shared-memory (akin to user-controlled cache), can take a lot of practice, so in the next exercise, we examine how well the simple shared-memory kernel from the NYU lab compares to the optimized codes provided by NVIDIA in the CuPy package. 

#### The tasks for this exercise:
1. Copy the matrix-matrix kernel codes from the NYU lab. Test them for accurracy against *numpy.dot()* and also compare time runtimes these GPU kernels the numpy.dot() function as well. **Note:** Use [CUDA events](https://numba.readthedocs.io/en/stable/cuda-reference/host.html#events) when timing GPU kernel calls because the driver does not "block" the calling process (for case this is IPython). Insted, the kernel is sent to the GPU to run, and then the process (IPython) immediately continues to it's next bit of code. Contrary to GPU kernel calls, calls to copy data to or from the GPU will block the process. For these cases, the calls can be timed the same way that other Python calls are timed.<br> **For both GPU kernels:**
    - Test with square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 5120$, $N=10240$, and $N=20480$. **Tip**, first make sure you can get the GPU codes to work and that you get correct results by testing with $N_{test}=32$.
    - For each $N$ above, test the multiplication for both dtypes: *dtype=float32* and *dtype=float64*.
    - Calculate and show the error between your functions and the *numpy.dot()* function. 
    - Calculate and show the *speedup* (or *slowdown*) of your GPU kernel for each $N$ vs *numpy.dot()*. Be sure to include the array copy times in the "total-gpu-kernel runtime.
    - For each $N$ vs, calculate and show the *speedup* of your GPU kernel using *dtype=float32* vs *dtype=float64*. Be sure to include the array copy times in the "total-gpu-kernel runtime."
    
<br>

2. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
h_A = np.random.random((N, N)).astype(np.<float-type>)
h_B = np.random.random((N, N)).astype(np.<float-type>)
```    
<br>

3. For the device memory:
    - Create **d_A** and **d_B** by copying **h_A** and **h_B** to the GPU, and be sure to time the copies
    - Create **d_C** as device-array that is allocated on the GPU (device) only, and not on the host (**Do Not Copy**)
    
<br>

4. After the GPU matrix-matrix multiplication kernel finishes, **copy** the the *device-array* **d_C** to the *host-array* **h_C**, and be sure to time this copy.

<br>

5. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the speedup or slowdowns vs numpy as well as float32 vs float64. Remember, that your runtime for the GPU kernel include time to compile the kernel (not much you can do to control this). Futhermore, becasue you have to copy data to and off of the GPU, these copy times should be included in the "total-gpu-kernel runtime." 

In [33]:
### Loops within CUDA kernel
# Matrix multiplication (NO shared memory)

@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    start_x, start_y = cuda.grid(2)
    stride_x, stride_y = cuda.gridsize(2)

    for i in range(start_x, C.shape[0], stride_x):
        for j in range(start_y, C.shape[1], stride_y):
            tmp = 0.
            for k in range(A.shape[1]):
                tmp += A[i, k] * B[k, j]
            C[i, j] = tmp

In [ ]:
### Loops within CUDA kernel
### Not used currently, shared memory case seems problematic
# Matrix multiplication (WITH shared memory)

# Float type
ftype = 'float32'

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPB * TPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=ftype)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=ftype)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    start_x, start_y = cuda.grid(2)
    stride_x, stride_y = cuda.gridsize(2)
    
    for x in range(start_x, C.shape[0], stride_x):
        for y in range(start_y, C.shape[1], stride_y):

            # Each thread computes one element in the result matrix.
            # The dot product is chunked into dot products of TPB-long vectors.
            tmp = 0.
            for i in range(int(A.shape[1] / TPB)):
                # Preload data into shared memory
                sA[tx, ty] = A[x, ty + i * TPB]
                sB[tx, ty] = B[tx + i * TPB, y]

                # Wait until all threads finish preloading
                cuda.syncthreads()

                # Computes partial product on the shared memory
                for j in range(TPB):
                    tmp += sA[tx, j] * sB[j, ty]

                # Wait until all threads finish computing
                cuda.syncthreads()

            C[x, y] = tmp

In [4]:
# Matrix multiplication (NO shared memory)

@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """

    # Check if (x, y) is within C
    x, y = cuda.grid(2)
    if x >= C.shape[0] or y >= C.shape[1]:
        return

    tmp = 0.
    for k in range(A.shape[1]):
        tmp += A[x, k] * B[k, y]
    C[x, y] = tmp

In [12]:
# Float type
ftype = 'float32'

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPB * TPB elements.
TPB = 16

# Matrix multiplication (WITH shared memory)
@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Check if (x, y) is within C
    x, y = cuda.grid(2)
    if x >= C.shape[0] or y >= C.shape[1]:
        return

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=ftype)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=ftype)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [13]:
# Matrix size
N = 5120

# Create matrices
h_A = np.random.random((N, N)).astype(ftype)
h_B = np.random.random((N, N)).astype(ftype)

In [14]:
# Np.dot function

# Start Cuda event timing
ev_start = cuda.event(timing=True)
ev_end = cuda.event(timing=True)
ev_start.record()

np_C = np.dot(h_A, h_B)

# End Cuda event timing
ev_end.record()
ev_end.synchronize()
t_np = cuda.event_elapsed_time(ev_start, ev_end)

In [15]:
# Matrix multiplication (NO shared memory)

# Configure the blocks
max_threads_per_block = 16

threads_per_block = np.min([max_threads_per_block, N])
blocks_per_grid = (N-1) // threads_per_block + 1

# Start Cuda event timing
ev_start = cuda.event(timing=True)
ev_end = cuda.event(timing=True)
ev_kernel_start = cuda.event(timing=True)
ev_kernel_end = cuda.event(timing=True)
ev_start.record()

# Copy to device
d_A = cuda.to_device(h_A)
d_B = cuda.to_device(h_B)

# Allocate memory on device for multiplication
d_C = cuda.device_array((N, N), dtype=ftype)

# Cuda kernel for multiplication
ev_kernel_start.record()
matmul[(blocks_per_grid, blocks_per_grid), (threads_per_block, threads_per_block)](d_A, d_B, d_C)
ev_kernel_end.record()

# Copy result back to host
h_C = d_C.copy_to_host()

# End Cuda event timing
ev_end.record()
ev_end.synchronize()
t_kernel = cuda.event_elapsed_time(ev_kernel_start, ev_kernel_end)
t_total = cuda.event_elapsed_time(ev_start, ev_end)

# Check result
print(f'Element-wise error: {np.mean(h_C - np_C):.6g}. Epsilon for current float type: {np.finfo(ftype).eps:.6g}.')
print(f'GPU kernel runtime: {(t_kernel/1e3):6.5f} s.')
print(f'Total GPU runtime: {(t_total/1e3):6.5f} s.')
print(f'Numpy runtime: {(t_np/1e3):6.5f} s.')

Element-wise error: 7.20378e-08. Epsilon for current float type: 1.19209e-07.
GPU kernel runtime: 3.11498 s.
Total GPU runtime: 3.32405 s.
Numpy runtime: 0.60341 s.


In [16]:
# Matrix multiplication (WITH shared memory)

# Configure the blocks
threads_per_block = TPB
blocks_per_grid = (N-1) // threads_per_block + 1

# Start Cuda event timing
ev_start = cuda.event(timing=True)
ev_end = cuda.event(timing=True)
ev_kernel_start = cuda.event(timing=True)
ev_kernel_end = cuda.event(timing=True)
ev_start.record()

# Copy to device
d_A = cuda.to_device(h_A)
d_B = cuda.to_device(h_B)

# Allocate memory on device for multiplication
d_C = cuda.device_array((N, N), dtype=ftype)

# Cuda kernel for transpose
ev_kernel_start.record()
fast_matmul[(blocks_per_grid, blocks_per_grid), (threads_per_block, threads_per_block)](d_A, d_B, d_C)
ev_kernel_end.record()

# Copy result back to host
h_C = d_C.copy_to_host()

# End Cuda event timing
ev_end.record()
ev_end.synchronize()
t_kernel = cuda.event_elapsed_time(ev_kernel_start, ev_kernel_end)
t_total = cuda.event_elapsed_time(ev_start, ev_end)

# Check result
print(f'Element-wise error: {np.mean(h_C - np_C):.6g}. Epsilon for current float type: {np.finfo(ftype).eps:.6g}.')
print(f'GPU kernel runtime: {(t_kernel/1e3):6.5f} s.')
print(f'Total GPU runtime: {(t_total/1e3):6.5f} s.')
print(f'Numpy runtime: {(t_np/1e3):6.5f} s.')

Element-wise error: 7.20378e-08. Epsilon for current float type: 1.19209e-07.
GPU kernel runtime: 2.67635 s.
Total GPU runtime: 2.82120 s.
Numpy runtime: 0.60341 s.


<br>

### Exercise 3: CuPy 

For this exercise, we will repeat what we did in *Exercise 2*. However, we will use *CuPy* functions, which are similar to *Numpy* funcstions with some added functions for copying data to-the-device-from-the-host and to-the-host-from-the-device. By using CuPy, we can depend on code that has been optimized for the GPU by NVIDIA, and instead of tyring to optimize our matrix-matrix multiplication kernels, we can use a built-in function to calculate the multiplication instead (i.e. [cupy.dot()](https://docs.cupy.dev/en/stable/reference/generated/cupy.dot.html#cupy.dot)).

**Tasks for this exercise:**
* Same as those listed in *Exercise 2*, but compare *cupy.dot()* to *numpy.dot()*.
* Also, reuse the host-arrays, *h_A* and *h_B* above. You will need to call the appropriate *CuPy* fuctions to copy these arrays to the GPU and to copy the result back to the host. You will **not** need to declare the deive-C array before calling *cupy.dot()* because the function will do it for you (like numpy does).


In [26]:
# Matrix size
N = 5120

# Float type
ftype = 'float64'

# Input matrix
h_A = np.random.random((N, N)).astype(ftype)
h_B = np.random.random((N, N)).astype(ftype)

In [27]:
# Np.dot function

# Start Cuda event timing
ev_start = cuda.event(timing=True)
ev_end = cuda.event(timing=True)
ev_start.record()

np_C = np.dot(h_A, h_B)

# End Cuda event timing
ev_end.record()
ev_end.synchronize()
t_np = cuda.event_elapsed_time(ev_start, ev_end)

In [28]:
# Cupy function

# Start Cuda event timing
ev_start = cuda.event(timing=True)
ev_end = cuda.event(timing=True)
ev_start.record()

# Cupy matrix multiplication
cu_A = cupy.asarray(h_A)
cu_B = cupy.asarray(h_B)
cu_C = cupy.dot(cu_A, cu_B)
h_C = cupy.asnumpy(cu_C)

# End Cuda event timing
ev_end.record()
ev_end.synchronize()
t_cp = cuda.event_elapsed_time(ev_start, ev_end)

print(f'Element-wise error: {np.mean(h_C - np_C):.6g}. Epsilon for current float type: {np.finfo(ftype).eps:.6g}.')
print(f'Cupy function time: {(t_cp/1e3):6.5f} s')
print(f'Numpy function time: {(t_np/1e3):6.5f} s')

Element-wise error: -1.26826e-16. Epsilon for current float type: 2.22045e-16.
Cupy function time: 1.50549 s
Numpy function time: 1.18491 s


<br>

### Exercise $\mathbf{\pi}$: CuPy Interoperability

Numba and CuPy device arrays (GPU arrays) can be accept each other's arrays. See [Interoperability](https://docs.cupy.dev/en/stable/user_guide/interoperability.html).

**Tasks for this exercise**
* Use the **device** arrays, **d_A** and **d_B**, that were created in *Exercise 2* to calculate the matrix-matrix multiplcation using *cupy.dot()*.
* Verify that you get the same results as you did in *Exercise 3*.
* You will need to "wrap" the device arrays before passing them to *cupy.dot()*. Read the *Interoperability* documentation linked above.
    - Time how long it takes (runtime) to "wrap" these arrays.
    - Compare this runtime to the runtime it took to create the device arrays in *Exercise 3*.
    - Provide a quick comment your thoughts on the runtime differences compared above.

In [29]:
# Matrix size
N = 5120

# Float type
ftype = 'float64'

# Input matrix
h_A = np.random.random((N, N)).astype(ftype)
h_B = np.random.random((N, N)).astype(ftype)

In [30]:
# Start timing
ev_start = cuda.event(timing=True)
ev_end = cuda.event(timing=True)
ev_start.record()

# Host to Cupy array
cu_A1 = cupy.asarray(h_A)
cu_B1 = cupy.asarray(h_B)

# End timing
ev_end.record()
ev_end.synchronize()
t_lapse = cuda.event_elapsed_time(ev_start, ev_end)

print(f'From Host to Cupy array: {t_lapse:6.5f} ms')

# Result as Exercise 3
cu_C1 = cupy.dot(cu_A1, cu_B1)
h_C1 = cupy.asnumpy(cu_C1)
del cu_A1, cu_B1, cu_C1

# Start timing
ev_start = cuda.event(timing=True)
ev_end = cuda.event(timing=True)
ev_start.record()

# Host to Device
d_A = cuda.to_device(h_A)
d_B = cuda.to_device(h_B)

# End timing
ev_end.record()
ev_end.synchronize()
t_lapse = cuda.event_elapsed_time(ev_start, ev_end)

print(f'From Host to Device: {t_lapse:6.5f} ms')

# Start timing
ev_start = cuda.event(timing=True)
ev_end = cuda.event(timing=True)
ev_start.record()

# Device to Cupy array
cu_A2 = cupy.asarray(d_A)
cu_B2 = cupy.asarray(d_B)

# End timing
ev_end.record()
ev_end.synchronize()
t_lapse = cuda.event_elapsed_time(ev_start, ev_end)

print(f'From Device to Cupy array: {t_lapse:6.5f} ms')

# New result
cu_C2 = cupy.dot(cu_A2, cu_B2)
h_C2 = cupy.asnumpy(cu_C2)
del cu_A2, cu_B2, cu_C2

# Check results
assert np.all(np.abs(h_C2 - h_C1)) == 0, 'Result not correct!'

From Host to Cupy array: 99.04585 ms
From Host to Device: 92.06969 ms
From Device to Cupy array: 0.42694 ms
